In [1]:
import pygame
import random
import math
import sys

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()

(5, 0)

In [3]:
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600
FPS = 60

In [4]:
# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 100, 255)
YELLOW = (255, 255, 0)
GRAY = (128, 128, 128)
DARK_GRAY = (64, 64, 64)
ORANGE = (255, 165, 0)
PURPLE = (128, 0, 128)

In [5]:
# Road constants
ROAD_WIDTH = 400
LANE_WIDTH = ROAD_WIDTH // 3
ROAD_LEFT = (WINDOW_WIDTH - ROAD_WIDTH) // 2
ROAD_RIGHT = ROAD_LEFT + ROAD_WIDTH

In [6]:
class Car:
    def __init__(self, x, y, color=BLUE, is_player=True):
        self.x = x
        self.y = y
        self.width = 30
        self.height = 50
        self.color = color
        self.is_player = is_player
        self.speed = 0
        self.max_speed = 8
        self.acceleration = 0.5
        self.deceleration = 0.3
        self.turn_speed = 0
        self.max_turn_speed = 5
        
        # Animation
        self.tire_offset = 0
        
    def update(self, keys_pressed=None):
        if self.is_player and keys_pressed:
            self.handle_input(keys_pressed)
        
        # Update tire animation
        if self.speed > 0:
            self.tire_offset = (self.tire_offset + self.speed * 0.5) % 20
            
    def handle_input(self, keys_pressed):
        # Acceleration and braking
        if keys_pressed[pygame.K_UP] or keys_pressed[pygame.K_w]:
            self.speed = min(self.speed + self.acceleration, self.max_speed)
        elif keys_pressed[pygame.K_DOWN] or keys_pressed[pygame.K_s]:
            self.speed = max(self.speed - self.acceleration * 1.5, -self.max_speed * 0.5)
        else:
            if self.speed > 0:
                self.speed = max(0, self.speed - self.deceleration)
            elif self.speed < 0:
                self.speed = min(0, self.speed + self.deceleration)
                
        # Turning (only when moving)
        if abs(self.speed) > 0.5:
            if keys_pressed[pygame.K_LEFT] or keys_pressed[pygame.K_a]:
                self.turn_speed = max(self.turn_speed - 0.5, -self.max_turn_speed)
            elif keys_pressed[pygame.K_RIGHT] or keys_pressed[pygame.K_d]:
                self.turn_speed = min(self.turn_speed + 0.5, self.max_turn_speed)
            else:
                self.turn_speed *= 0.8  # Gradual return to center
                
            # Apply turning (proportional to speed)
            turn_factor = abs(self.speed) / self.max_speed
            self.x += self.turn_speed * turn_factor
            
            # Keep car on road
            self.x = max(ROAD_LEFT + 10, min(ROAD_RIGHT - self.width - 10, self.x))
            
    def get_rect(self):
        return pygame.Rect(self.x, self.y, self.width, self.height)
        
    def draw(self, screen):
        car_rect = pygame.Rect(self.x, self.y, self.width, self.height)
        
        # Main car body
        pygame.draw.rect(screen, self.color, car_rect)
        pygame.draw.rect(screen, WHITE, car_rect, 2)
        
        # Windshield
        windshield = pygame.Rect(self.x + 5, self.y + 5, self.width - 10, 15)
        pygame.draw.rect(screen, WHITE, windshield)
        
        # Tires
        tire_color = BLACK
        # Front tires
        pygame.draw.rect(screen, tire_color, (self.x - 3, self.y + 5, 6, 12))
        pygame.draw.rect(screen, tire_color, (self.x + self.width - 3, self.y + 5, 6, 12))
        # Rear tires
        pygame.draw.rect(screen, tire_color, (self.x - 3, self.y + self.height - 17, 6, 12))
        pygame.draw.rect(screen, tire_color, (self.x + self.width - 3, self.y + self.height - 17, 6, 12))
        
        # Headlights (for player car)
        if self.is_player:
            pygame.draw.circle(screen, YELLOW, (int(self.x + 8), int(self.y)), 4)
            pygame.draw.circle(screen, YELLOW, (int(self.x + self.width - 8), int(self.y)), 4)


In [7]:
class Obstacle:
    def __init__(self, x, y, obstacle_type="car"):
        self.x = x
        self.y = y
        self.type = obstacle_type
        self.speed = random.uniform(2, 4)
        
        if obstacle_type == "car":
            self.width = 30
            self.height = 50
            self.color = random.choice([RED, GREEN, ORANGE, PURPLE])
        elif obstacle_type == "oil":
            self.width = 25
            self.height = 25
            self.color = BLACK
        elif obstacle_type == "debris":
            self.width = 20
            self.height = 30
            self.color = GRAY
            
    def update(self, game_speed):
        self.y += self.speed + game_speed
        
    def get_rect(self):
        return pygame.Rect(self.x, self.y, self.width, self.height)
        
    def draw(self, screen):
        if self.type == "car":
            car = Car(self.x, self.y, self.color, False)
            car.draw(screen)
        elif self.type == "oil":
            # Oil spill
            pygame.draw.ellipse(screen, self.color, self.get_rect())
            pygame.draw.ellipse(screen, DARK_GRAY, self.get_rect(), 2)
        elif self.type == "debris":
            # Road debris
            pygame.draw.rect(screen, self.color, self.get_rect())
            pygame.draw.rect(screen, WHITE, self.get_rect(), 1)


In [8]:
class PowerUp:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.width = 25
        self.height = 25
        self.speed = 3
        self.color = YELLOW
        self.glow = 0
        
    def update(self, game_speed):
        self.y += self.speed + game_speed
        self.glow = (self.glow + 5) % 360
        
    def get_rect(self):
        return pygame.Rect(self.x, self.y, self.width, self.height)
        
    def draw(self, screen):
        # Pulsing effect
        pulse = abs(math.sin(math.radians(self.glow))) * 50 + 205
        color = (255, 255, int(pulse))
        
        # Star shape
        center_x = self.x + self.width // 2
        center_y = self.y + self.height // 2
        
        points = []
        for i in range(10):
            angle = math.radians(i * 36)
            radius = 12 if i % 2 == 0 else 6
            x = center_x + radius * math.cos(angle)
            y = center_y + radius * math.sin(angle)
            points.append((x, y))
            
        pygame.draw.polygon(screen, color, points)
        pygame.draw.polygon(screen, WHITE, points, 2)



In [9]:
class RacingGame:
    def __init__(self):
        self.screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
        pygame.display.set_caption("Retro Racing Game")
        self.clock = pygame.time.Clock()
        self.font = pygame.font.Font(None, 36)
        self.big_font = pygame.font.Font(None, 72)
        
        self.reset_game()
        
    def reset_game(self):
        # Player car
        self.player = Car(WINDOW_WIDTH // 2 - 15, WINDOW_HEIGHT - 100, BLUE)
        
        # Game state
        self.obstacles = []
        self.power_ups = []
        self.score = 0
        self.high_score = getattr(self, 'high_score', 0)
        self.game_speed = 3
        self.distance = 0
        self.time_elapsed = 0
        self.game_over = False
        self.game_started = False
        self.paused = False
        
        # Road animation
        self.road_offset = 0
        
        # Timers
        self.obstacle_timer = 0
        self.powerup_timer = 0
        self.speed_boost_timer = 0
        
    def spawn_obstacle(self):
        lane = random.choice([0, 1, 2])  # Three lanes
        x = ROAD_LEFT + lane * LANE_WIDTH + (LANE_WIDTH - 30) // 2
        y = -60
        obstacle_type = random.choices(
            ["car", "oil", "debris"], 
            weights=[70, 15, 15]
        )[0]
        self.obstacles.append(Obstacle(x, y, obstacle_type))
        
    def spawn_powerup(self):
        lane = random.choice([0, 1, 2])
        x = ROAD_LEFT + lane * LANE_WIDTH + (LANE_WIDTH - 25) // 2
        y = -30
        self.power_ups.append(PowerUp(x, y))
        
    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    return False
                elif event.key == pygame.K_SPACE:
                    if not self.game_started:
                        self.game_started = True
                    elif self.game_over:
                        self.reset_game()
                        self.game_started = True
                    else:
                        self.paused = not self.paused
        return True
        
    def update(self):
        if not self.game_started or self.game_over or self.paused:
            return
            
        keys = pygame.key.get_pressed()
        
        # Update player
        self.player.update(keys)
        
        # Update timers
        self.time_elapsed += 1/60
        self.obstacle_timer += 1
        self.powerup_timer += 1
        
        # Update road animation
        self.road_offset = (self.road_offset + self.game_speed) % 40
        
        # Spawn obstacles
        spawn_rate = max(30, 90 - self.distance // 100)  # Spawn more frequently as game progresses
        if self.obstacle_timer >= spawn_rate:
            self.spawn_obstacle()
            self.obstacle_timer = 0
            
        # Spawn power-ups occasionally
        if self.powerup_timer >= random.randint(300, 600):
            self.spawn_powerup()
            self.powerup_timer = 0
            
        # Update obstacles
        for obstacle in self.obstacles[:]:
            obstacle.update(self.game_speed)
            if obstacle.y > WINDOW_HEIGHT:
                self.obstacles.remove(obstacle)
                self.score += 10  # Points for dodging
                
        # Update power-ups
        for powerup in self.power_ups[:]:
            powerup.update(self.game_speed)
            if powerup.y > WINDOW_HEIGHT:
                self.power_ups.remove(powerup)
                
        # Check collisions
        player_rect = self.player.get_rect()
        
        # Obstacle collisions
        for obstacle in self.obstacles:
            if player_rect.colliderect(obstacle.get_rect()):
                if obstacle.type == "oil":
                    # Oil spill - lose control temporarily
                    self.player.turn_speed += random.uniform(-3, 3)
                    self.obstacles.remove(obstacle)
                else:
                    # Crash
                    self.game_over = True
                    if self.score > self.high_score:
                        self.high_score = self.score
                        
        # Power-up collisions
        for powerup in self.power_ups[:]:
            if player_rect.colliderect(powerup.get_rect()):
                self.score += 50
                self.speed_boost_timer = 180  # 3 seconds at 60 FPS
                self.power_ups.remove(powerup)
                
        # Apply speed boost
        if self.speed_boost_timer > 0:
            self.game_speed = min(8, self.game_speed + 0.1)
            self.speed_boost_timer -= 1
        else:
            # Gradually increase game speed
            self.game_speed = min(7, 3 + self.distance / 1000)
            
        # Update distance and score
        self.distance += self.game_speed
        self.score += int(self.game_speed * 0.1)  # Distance bonus
        
    def draw_road(self):
        # Road background
        road_rect = pygame.Rect(ROAD_LEFT, 0, ROAD_WIDTH, WINDOW_HEIGHT)
        pygame.draw.rect(self.screen, DARK_GRAY, road_rect)
        
        # Lane dividers (animated)
        divider_width = 4
        divider_height = 20
        divider_gap = 20
        
        for lane in range(1, 3):  # Two dividers for three lanes
            x = ROAD_LEFT + lane * LANE_WIDTH - divider_width // 2
            y = -divider_height + self.road_offset
            
            while y < WINDOW_HEIGHT:
                divider_rect = pygame.Rect(x, y, divider_width, divider_height)
                pygame.draw.rect(self.screen, YELLOW, divider_rect)
                y += divider_height + divider_gap
                
        # Road borders
        pygame.draw.line(self.screen, WHITE, (ROAD_LEFT, 0), (ROAD_LEFT, WINDOW_HEIGHT), 4)
        pygame.draw.line(self.screen, WHITE, (ROAD_RIGHT, 0), (ROAD_RIGHT, WINDOW_HEIGHT), 4)
        
    def draw_hud(self):
        # Score
        score_text = self.font.render(f"Score: {self.score}", True, WHITE)
        self.screen.blit(score_text, (10, 10))
        
        # High Score
        high_score_text = self.font.render(f"High Score: {self.high_score}", True, WHITE)
        self.screen.blit(high_score_text, (10, 50))
        
        # Speed
        speed_text = self.font.render(f"Speed: {int(self.game_speed * 10)} mph", True, WHITE)
        self.screen.blit(speed_text, (WINDOW_WIDTH - 200, 10))
        
        # Distance
        distance_text = self.font.render(f"Distance: {int(self.distance)}m", True, WHITE)
        self.screen.blit(distance_text, (WINDOW_WIDTH - 200, 50))
        
        # Time
        time_text = self.font.render(f"Time: {int(self.time_elapsed)}s", True, WHITE)
        self.screen.blit(time_text, (WINDOW_WIDTH - 200, 90))
        
        # Speed boost indicator
        if self.speed_boost_timer > 0:
            boost_text = self.font.render("SPEED BOOST!", True, YELLOW)
            boost_rect = boost_text.get_rect(center=(WINDOW_WIDTH // 2, 50))
            self.screen.blit(boost_text, boost_rect)
            
    def draw_start_screen(self):
        overlay = pygame.Surface((WINDOW_WIDTH, WINDOW_HEIGHT))
        overlay.set_alpha(180)
        overlay.fill(BLACK)
        self.screen.blit(overlay, (0, 0))
        
        title_text = self.big_font.render("RETRO RACING", True, WHITE)
        title_rect = title_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 - 100))
        self.screen.blit(title_text, title_rect)
        
        instructions = [
            "ARROW KEYS or WASD to steer and accelerate",
            "Avoid obstacles and collect power-ups!",
            "SPACE to start/pause",
            "",
            "Press SPACE to start racing!"
        ]
        
        for i, instruction in enumerate(instructions):
            if instruction:
                color = GREEN if "Press SPACE" in instruction else WHITE
                text = self.font.render(instruction, True, color)
                text_rect = text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 - 20 + i * 35))
                self.screen.blit(text, text_rect)
                
    def draw_pause_screen(self):
        overlay = pygame.Surface((WINDOW_WIDTH, WINDOW_HEIGHT))
        overlay.set_alpha(128)
        overlay.fill(BLACK)
        self.screen.blit(overlay, (0, 0))
        
        pause_text = self.big_font.render("PAUSED", True, WHITE)
        pause_rect = pause_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2))
        self.screen.blit(pause_text, pause_rect)
        
        instruction_text = self.font.render("Press SPACE to resume", True, GREEN)
        instruction_rect = instruction_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 + 60))
        self.screen.blit(instruction_text, instruction_rect)
        
    def draw_game_over_screen(self):
        overlay = pygame.Surface((WINDOW_WIDTH, WINDOW_HEIGHT))
        overlay.set_alpha(180)
        overlay.fill(BLACK)
        self.screen.blit(overlay, (0, 0))
        
        game_over_text = self.big_font.render("GAME OVER", True, RED)
        game_over_rect = game_over_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 - 80))
        self.screen.blit(game_over_text, game_over_rect)
        
        final_score_text = self.font.render(f"Final Score: {self.score}", True, WHITE)
        final_score_rect = final_score_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 - 20))
        self.screen.blit(final_score_text, final_score_rect)
        
        distance_text = self.font.render(f"Distance Traveled: {int(self.distance)}m", True, WHITE)
        distance_rect = distance_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 + 10))
        self.screen.blit(distance_text, distance_rect)
        
        time_text = self.font.render(f"Time Survived: {int(self.time_elapsed)}s", True, WHITE)
        time_rect = time_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 + 40))
        self.screen.blit(time_text, time_rect)
        
        if self.score == self.high_score and self.score > 0:
            new_record_text = self.font.render("NEW HIGH SCORE!", True, GREEN)
            new_record_rect = new_record_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 + 80))
            self.screen.blit(new_record_text, new_record_rect)
            
        restart_text = self.font.render("Press SPACE to race again", True, GREEN)
        restart_rect = restart_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 + 120))
        self.screen.blit(restart_text, restart_rect)
        
    def draw(self):
        self.screen.fill(GREEN)  # Grass background
        
        if self.game_started:
            self.draw_road()
            
            # Draw game objects
            for obstacle in self.obstacles:
                obstacle.draw(self.screen)
                
            for powerup in self.power_ups:
                powerup.draw(self.screen)
                
            self.player.draw(self.screen)
            self.draw_hud()
            
        # Draw overlays
        if not self.game_started:
            self.draw_start_screen()
        elif self.paused:
            self.draw_pause_screen()
        elif self.game_over:
            self.draw_game_over_screen()
            
        pygame.display.flip()
        
    def run(self):
        running = True
        while running:
            running = self.handle_events()
            self.update()
            self.draw()
            self.clock.tick(FPS)
            
        pygame.quit()
        sys.exit()

if __name__ == "__main__":
    game = RacingGame()
    game.run()

SystemExit: 

C:\Users\anas0\anaconda5\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
